In [44]:
#
#
#
#

from urllib import request
from time   import sleep
from bs4    import BeautifulSoup

import requests
import pandas as pd

# Links for IOP search

IOP_S  = "https://iopscience.iop.org/nsearch?terms="
IOP_E  = "&nextPage=2&previousPage=-1&currentPage="
IOP_P  = "&searchDatePeriod=anytime&orderBy=relevance&pageLength=50"

# Links for NATURE search

NANO_S = 'https://nano.nature.com/search?term=freeText%3A'
NANO_E = '&sort-by=relevance&page-number='
NANO_P = '&workflow=article'

# /Souparticles : Returns a list of BeautifulSoup objects that contain articles

# /material     : Is a str. containing the search term
# /page         : The page in which we are searching
# /max_try      : The maximum tries before givin up, commonly this is a result of a 400 status response from the server
# /slpy         : Is the time between requests, recommended to prevent overloading requested server


# Some pages can show problems if requested with -urllib- or -requests-, SwitchKitchen is implemented to
# switch the way we obtain a BeautifulSoup object

def SwitchKitchen(uri, kitchen = "requests", parse = "lxml"):
    if   kitchen == "requests":
         response = requests.get(uri)
         soup     = BeautifulSoup(response.text, 'lxml')
    elif kitchen == "urllib":
         response = request.urlopen(uri)
         soup     = BeautifulSoup(response, 'lxml')
    else:
        raise Exception("Problema en la elección del método mara obtener la respuesta del servidor.")
    return soup;

def souparticlesIOP(query, page, max_int = 5, slpy = 3):
    # At the beginning our IOP Soup is a None object
    soupIOP = None
    ints  = 0

    while ints<max_int:    
    # We try to catch some content from the server

        try:
            uri      = IOP_S+ query + IOP_E + str(page) + IOP_P
            # If we catch a 200 response we can make a Beautifoul Soup (BS), parsing with lxml
            soupIOP  = SwitchKitchen(uri,kitchen = "urllib")
            break
        except:
            # If we catch an error, we can retry
            print("Reintentando pag: " + str(page))
            sleep(slpy)
            pass
        finally:
            ints +=1
    
    # For soupIOP == None this process needs to be repeated
    if(soupIOP==None):
        print( "Por favor reinicie la busqueda para pag: " + str(page))
        return None;
    else:
        # Returning the articles in a BS object
        print("Añadida pag: " + str(page))
        return soupIOP.find_all('div', {"class" : "art-list-item-body"})


def souparticlesNANO(query, page, max_int = 5, slpy = 3):
    nanosoup = None
    ints=0
    while ints<max_int:
        try:
            uri      = NANO_S+'"'+query+'"'+NANO_E+str(page)+NANO_P
            nanosoup = SwitchKitchen(uri, kitchen = "requests")
            break
        except:
            print("Reintentando pag: " + str(page))
            sleep(slpy)
            pass
        finally:
            ints +=1
    if(nanosoup==None):
        print( "Por favor reinicie la busqueda para pag: " + str(page))
        return None;
    else:
        print("Añadida pag: " + str(page))
        return nanosoup.find_all('li', {"class" : "Results_listItem" })

def PagesIOP(query, end=12):
    all_     = []
    to_retry = []
    for i in range(1,end):
        item = souparticlesIOP(query, i)
        if(isinstance(item, str)):
            to_retry.append(i)
        else:
            all_.append(item)
    return [all_, to_retry];

def PagesNANO(query, end=12):
    all_     = []
    to_retry = []
    for i in range(1,end):
        item = souparticlesNANO(query, i)
        if(isinstance(item, str)):
            to_retry.append(i)
        else:
            all_.append(item)
    return [all_, to_retry];

def stringify(raw_pages):
    arts_db = []
    for page in raw_pages:
        for element in page:
            try:
                title     =     element.findChild('h2' , {"class"   : "art-list-item-title"}).findChild("a")          # 0 title
                abstract  =     element.findChild('div', {"class"   : "article-text view-text-small"}).findChild("p") # 1 abstract
                DOI       =     element.findChild('a'  , {"class"   : "mr-2"})                                        # 2 DOI
                journal   =     element.findChild('em')                                                               # 3 Journal
                vol       =     element.findChild('b')                                                                # 4 vol
                authors   =     element.find_all('span', {"itemprop": "author"})                                      # 5 auths *beta*

                yearfind  =     element.find_all("p"   , { "class" :"small art-list-item-meta"})                      # 6 year
                year      =     int(yearfind[1].text[14:19])

                arts_db.append([title.text ,abstract.text, DOI.text, journal.text, vol.text ,[auth.getText() for auth in authors], year])
            except:
                pass

    return arts_db;


In [45]:
a1 = souparticlesIOP("ZnTe", 1)
a2 = souparticlesNANO("ZnTe", 1)

Añadida pag: 1
Añadida pag: 1


In [47]:
len(a1)

50

In [5]:
def panda_to_me(all_arts):
    test_arts = {}
    for i in range(0,7):
        ls = []
        for j in range(0,len(all_arts)):
            ls.append(all_arts[j][i])
        if i == 0:
            test_arts.update(Title        = ls)
        elif i == 1:
            test_arts.update(Abstract     = ls)
        elif i == 2:
            test_arts.update(DOI          = ls)
        elif i == 3:
            test_arts.update(Journal      = ls)
        elif i == 4:
            test_arts.update(Vol          = ls)
        elif i == 5:
            test_arts.update(Auths        = ls)
        else:
            test_arts.update(Year         = ls)
    return pd.DataFrame(test_arts);

In [8]:
ZnTe_db    = stringify(ZnTe)

In [9]:
ZnTe_panda = panda_to_me(ZnTe_db)

In [11]:
ZnTe_panda.to_pickle("mispepinillos/ZnTe_DB.pkl")